# Convolutional Neural Networks (CNN)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Generate a more complex dataset with 3 classes
X, y = make_classification(
    n_samples=200,
    n_features=2,
    n_classes=3,
    n_informative=2,
    n_redundant=0,
    n_clusters_per_class=1,
    class_sep=0.5,
    flip_y=0.1,
    random_state=42
)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the dataset
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Reshape input for CNN: (batch_size, channels, height, width)
# For this example, we add a channel dimension (1) and treat the 2D data as a 1x2 "image"
X_train_tensor = X_train_tensor.unsqueeze(1)  # Add channel dimension, making it (batch_size, 1, 2)
X_test_tensor = X_test_tensor.unsqueeze(1)    # Same for test set

# Define the CNN Model
class CNN(nn.Module):
    def __init__(self, input_channels, output_size):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 16, kernel_size=2, stride=1, padding=1)  # First convolution layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)  # Pooling layer
        self.fc1 = nn.Linear(16 * 1 * 1, output_size)  # Fully connected layer

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))  # Apply conv -> relu -> pooling
        x = x.view(-1, 16 * 1 * 1)  # Flatten the output of conv layer for FC layer
        x = self.fc1(x)
        return x

# Initialize the CNN model
input_channels = 1  # Since we're treating the 2D data as 1 channel "image"
output_size = 3  # 3 classes

cnn_model = CNN(input_channels, output_size)

# Choose optimizer and loss function
criterion = nn.CrossEntropyLoss()
cnn_optimizer = optim.Adam(cnn_model.parameters(), lr=0.01)

# Train the CNN Model
def train_cnn_model(model, optimizer, X_train_tensor, y_train_tensor, epochs=100):
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch}/{epochs}, Loss: {loss.item():.4f}")

# Train CNN model
print("Training CNN Model...")
train_cnn_model(cnn_model, cnn_optimizer, X_train_tensor, y_train_tensor)

# Visualizing the decision boundary
def plot_decision_boundary(X, y, model, num_classes, cnn=False):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
    
    grid = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)
    grid = grid.unsqueeze(1)  # Add channel dimension
    
    model.eval()
    with torch.no_grad():
        Z = model(grid)
        Z = torch.argmax(Z, axis=1).numpy()
    
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, alpha=0.2, colors=['red', 'blue', 'green'], levels=np.arange(-0.5, 3, 1))
    plt.contour(xx, yy, Z, levels=np.arange(-0.5, 3, 1), colors='black', linewidths=1)

    colors = ['red', 'blue', 'green']
    for i in range(num_classes):
        plt.scatter(X[y == i, 0], X[y == i, 1], color=colors[i], label=f"Class {i}", edgecolor='k', s=100)
    
    plt.title("Decision Boundary with CNN Model", fontsize=14)
    plt.xlabel("Feature 1", fontsize=12)
    plt.ylabel("Feature 2", fontsize=12)
    plt.legend()
    plt.grid(True)
    plt.show()

# Plot decision boundaries for CNN model
print("Plotting decision boundary for CNN...")
plot_decision_boundary(X, y, cnn_model, num_classes=3, cnn=True)


Training CNN Model...


RuntimeError: Given groups=1, weight of size [16, 1, 2, 2], expected input[1, 160, 1, 2] to have 1 channels, but got 160 channels instead